In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/netflix-stock-price-data-set-20022022/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
!pip install pyspark

/kaggle/input/netflix-stock-price-data-set-20022022/NFLX.csv
     |████████████████████████████████| 281.4 MB 35 kB/s               
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=57195a0e51de97f26114e06aa21042d637d952e1c803b570d57126701688012d
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [3]:
sc=SparkContext(master="local")
spark=SparkSession(sc)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/19 02:41:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.catalog.listTables() # check whether there are tables in spark session

[]

In [5]:
pd_temp = pd.DataFrame(np.random.random(10)) # this is a pandas dataframe
spark_df=spark.createDataFrame(pd_temp) # spark dataframe image created locally 
spark.catalog.listTables() # as created locally will not find the spark dataframe

[]

In [6]:
spark_df.createOrReplaceTempView('random') # created a temp copy from local copy 
spark.catalog.listTables() # now one can see the spark dataframe in session

[Table(name='random', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [7]:
df_sql_spark=spark.sql("select * from random") # using spark.sql to fetch results from spark dataframe
df_pandas=df_sql_spark.toPandas() # spark dataframe converted to pandas dataframe

In [8]:
# directly importing csv file into spark
path="/kaggle/input/netflix-stock-price-data-set-20022022/NFLX.csv"
netf=spark.read.csv(path,header=True)
netf.show(2)
netf.createOrReplaceTempView("netf") # creating from local copy

+----------+--------+--------+--------+--------+---------+---------+
|      Date|    Open|    High|     Low|   Close|Adj Close|   Volume|
+----------+--------+--------+--------+--------+---------+---------+
|2002-05-23|1.156429|1.242857|1.145714|1.196429| 1.196429|104790000|
|2002-05-24|1.214286|1.225000|1.197143|1.210000| 1.210000| 11104800|
+----------+--------+--------+--------+--------+---------+---------+
only showing top 2 rows



In [9]:
netf_df=spark.table("netf")
netf_df=netf_df.withColumn('volmn',netf_df.Volume/1000000)
netf_df.show(3)

+----------+--------+--------+--------+--------+---------+---------+-------+
|      Date|    Open|    High|     Low|   Close|Adj Close|   Volume|  volmn|
+----------+--------+--------+--------+--------+---------+---------+-------+
|2002-05-23|1.156429|1.242857|1.145714|1.196429| 1.196429|104790000| 104.79|
|2002-05-24|1.214286|1.225000|1.197143|1.210000| 1.210000| 11104800|11.1048|
|2002-05-28|1.213571|1.232143|1.157143|1.157143| 1.157143|  6609400| 6.6094|
+----------+--------+--------+--------+--------+---------+---------+-------+
only showing top 3 rows



In [10]:
# Filtering data
netf_df.filter(netf_df.Low>4)

DataFrame[Date: string, Open: string, High: string, Low: string, Close: string, Adj Close: string, Volume: string, volmn: double]

In [11]:
# Selecting data
netf_df.select(netf_df.Low,netf_df.Close)

DataFrame[Low: string, Close: string]

In [12]:
#aggregating data
#netf_df.select(netf_df.Low,netf_df.Close,netf_df.volmn>10).filter(netf_df.Close>10).groupBy().sum("volmn")